## Exploring the company match dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [3]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [4]:
from autolabel import get_data

get_data('company')

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100% [........................................................] 474657 / 474657

100% [......................................................] 4702697 / 4702697

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [13]:
import json

from autolabel import LabelingAgent

In [14]:
# load the config
with open('config_company_match.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `entity_matching` (since it's an entity matching task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are provided with descriptions of companies from their websites...` (how we describe the task to the LLM)
* `prompt.labels`: `['not duplicate', 'duplicate']` (the full list of labels to choose from)
* `prompt.few_shot_num`: 3 (how many labeled examples to provide to the LLM)

In [16]:
config

{'task_name': 'CompanyEntityMatch',
 'task_type': 'entity_matching',
 'dataset': {'label_column': 'label', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are provided with descriptions of companies from their websites, and wikipedia pages. Your job is to categorize whether the descriptions are about the same company (duplicate) or different companies (not duplicate). Your answer must be from one of the following options:\n{labels}',
  'labels': ['not duplicate', 'duplicate'],
  'example_template': 'Company 1 description: {entity1}\nCompany 2 description: {entity2}\nDuplicate or not: {label}',
  'few_shot_examples': [{'entity1': 'lac wisconsin branding 95 1 & 96 1 the rock frequency 96.1 mhz translator s 95.1 w236ag fond du lac first air date 1965 as wcwc fm at 95.9 format mainstream rock erp 4 000 watts haat 123 meters 404 ft class a facility id 54510 transmitter coordinates 43 49 10.00 n 88 43 20.00 w 43.8194444 n 88.7

In [17]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [18]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset
ds = AutolabelDataset("data/company/test.csv", config=config)
agent.plan(ds)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $12.172 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0061 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are provided with descriptions of companies from their websites, and wikipedia pages. Your job is to categorize whether the descriptions are about the same company (duplicate) or different companies (not duplicate). Your answer must be from one of the following options:
not duplicate
duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Company 1 description: lac wisconsin branding 95 1 & 96 1 the rock frequency 96.1 mhz translator s 95.1 w236ag fond du lac first air date 1965 as wcwc fm at 95.9 format mainstream rock erp 4 000 watts haat 123 meters 404 ft class a facility id 54510 transmitter coordinates 43 49 10.00 n 88 43 20.00 w 43.8194444 n 88.7222222 w 43.8194444 ; 88.7222222 coordinates 43 49 10.00 n 88 43 20.00 w 43.8194444 n 88.7222222 w 43.8194444 ; 88.7222222 former callsigns wcwc fm 1965 1980 wyur 1980 1994 former frequencies 95.9 mhz 1965 affiliations cbs radio network westwood one pr

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [19]:
# now, do the actual labeling
ds = agent.run(ds, max_items=100)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 12:41:08 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 86846 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-06-14 12:41:08 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 86846 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-06-14 12:41:10 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens p

2023-06-14 12:41:27 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89084 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-06-14 12:41:27 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89084 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-06-14 12:41:28 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens p

2023-06-14 12:41:44 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88966 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-06-14 12:41:44 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88966 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-06-14 12:41:45 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens p

...

2023-06-14 12:42:20 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88702 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-06-14 12:42:20 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88702 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-06-14 12:42:21 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens p

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Actual Cost: 0.2862


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.87     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at 87% accuracy when labeling the first 100 examples. Let's see if we can use confidence scores to improve accuracy further by removing the less confident examples from our labeled set.

### Compute confidence scores


In [21]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'xxxxxxxxxxxxxxxxx'

In [23]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [24]:
agent = LabelingAgent(config=config)

In [25]:
from autolabel import AutolabelDataset
ds = AutolabelDataset("data/company/test.csv", config=config)
agent.plan(ds)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $12.172 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0061 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are provided with descriptions of companies from their websites, and wikipedia pages. Your job is to categorize whether the descriptions are about the same company (duplicate) or different companies (not duplicate). Your answer must be from one of the following options:
not duplicate
duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Company 1 description: lac wisconsin branding 95 1 & 96 1 the rock frequency 96.1 mhz translator s 95.1 w236ag fond du lac first air date 1965 as wcwc fm at 95.9 format mainstream rock erp 4 000 watts haat 123 meters 404 ft class a facility id 54510 transmitter coordinates 43 49 10.00 n 88 43 20.00 w 43.8194444 n 88.7222222 w 43.8194444 ; 88.7222222 coordinates 43 49 10.00 n 88 43 20.00 w 43.8194444 n 88.7222222 w 43.8194444 ; 88.7222222 former callsigns wcwc fm 1965 1980 wyur 1980 1994 former frequencies 95.9 mhz 1965 affiliations cbs radio network westwood one pr

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [26]:
ds = agent.run(ds, max_items=500)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 12:51:22 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 0.5763
Actual Cost: 1.6423


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 500     │ -inf      │ 0.894    │ 1.0             │
│ 1       │ 0.9988    │ 1.0      │ 0.002           │
│ 2       │ 0.9987    │ 0.5      │ 0.004           │
│ 9       │ 0.9946    │ 0.8889   │ 0.018           │
│ 10      │ 0.9937    │ 0.8      │ 0.02            │
│ 14      │ 0.9927    │ 0.8571   │ 0.028           │
│ 15      │ 0.9924    │ 0.8      │ 0.03            │
│ 39      │ 0.9856    │ 0.9231   │ 0.078           │
│ 41      │ 0.9854    │ 0.878    │ 0.082           │
│ 53      │ 0.9805    │ 0.9057   │ 0.106           │
│ 54      │ 0.9802    │ 0.8889   │ 0.108           │
│ 59      │ 0.9794    │ 0.8983   │ 0.118           │
│ 60      │ 0.9792    │ 0.8833   │ 0.12            │
│ 61      │ 0.9791    │ 0.8852   │ 0.122           │
│ 62      │ 0.979     │ 0.871    │ 0.124           │
│ 63      │ 0.979     │ 0.873    │ 0.126           │
│ 64      │ 0.9789    │ 0.8594   │ 0.128           │
│ 72      │ 0.9763    │ 0.875    │ 0.144           │
│ 73      │ 0.9762    │ 0.863    │ 0.146           │
│ 85      │ 0.971     │ 0.8824   │ 0.17            │
│ 86      │ 0.9701    │ 0.8721   │ 0.172           │
│ 95      │ 0.9669    │ 0.8842   │ 0.19            │
│ 96      │ 0.9666    │ 0.875    │ 0.192           │
│ 102     │ 0.9649    │ 0.8824   │ 0.204           │
│ 103     │ 0.9648    │ 0.8738   │ 0.206           │
│ 104     │ 0.9647    │ 0.875    │ 0.208           │
│ 105     │ 0.9646    │ 0.8667   │ 0.21            │
│ 117     │ 0.9585    │ 0.8803   │ 0.234           │
│ 118     │ 0.958     │ 0.8729   │ 0.236           │
│ 141     │ 0.9507    │ 0.8936   │ 0.282           │
│ 142     │ 0.9506    │ 0.8873   │ 0.284           │
│ 147     │ 0.9495    │ 0.8912   │ 0.294           │
│ 148     │ 0.9492    │ 0.8851   │ 0.296           │
│ 149     │ 0.9486    │ 0.8859   │ 0.298           │
│ 150     │ 0.9482    │ 0.88     │ 0.3             │
│ 157     │ 0.9422    │ 0.8854   │ 0.314           │
│ 158     │ 0.9412    │ 0.8797   │ 0.316           │
│ 196     │ 0.916     │ 0.9031   │ 0.392           │
│ 197     │ 0.9159    │ 0.8985   │ 0.394           │
│ 219     │ 0.899     │ 0.9087   │ 0.438           │
│ 220     │ 0.8979    │ 0.9045   │ 0.44            │
│ 260     │ 0.8786    │ 0.9192   │ 0.52            │
│ 261     │ 0.8785    │ 0.9157   │ 0.522           │
│ 310     │ 0.8319    │ 0.929    │ 0.62            │
│ 311     │ 0.8315    │ 0.926    │ 0.622           │
│ 317     │ 0.8285    │ 0.9274   │ 0.634           │
│ 318     │ 0.8282    │ 0.9245   │ 0.636           │
│ 352     │ 0.7858    │ 0.9318   │ 0.704           │
│ 353     │ 0.784     │ 0.9292   │ 0.706           │
│ 363     │ 0.7678    │ 0.9311   │ 0.726           │
│ 364     │ 0.7674    │ 0.9286   │ 0.728           │
│ 370     │ 0.7656    │ 0.9297   │ 0.74            │
│ 371     │ 0.7624    │ 0.9272   │ 0.742           │
│ 374     │ 0.7582    │ 0.9278   │ 0.748           │
│ 375     │ 0.758     │ 0.9253   │ 0.75            │
│ 384     │ 0.7481    │ 0.9271   │ 0.768           │
│ 385     │ 0.7469    │ 0.9247   │ 0.77            │
│ 386     │ 0.7462    │ 0.9249   │ 0.772           │
│ 387     │ 0.7448    │ 0.9225   │ 0.774           │
│ 399     │ 0.7344    │ 0.9248   │ 0.798           │
│ 400     │ 0.7336    │ 0.9225   │ 0.8             │
│ 401     │ 0.7335    │ 0.9227   │ 0.802           │
│ 403     │ 0.7301    │ 0.9181   │ 0.806           │
│ 405     │ 0.7277    │ 0.9185   │ 0.81            │
│ 406     │ 0.7276    │ 0.9163   │ 0.812           │
│ 412     │ 0.7111    │ 0.9175   │ 0.824           │
│ 414     │ 0.7092    │ 0.913    │ 0.828           │
│ 415     │ 0.7078    │ 0.9133   │ 0.83            │
│ 417     │ 0.7041    │ 0.9089   │ 0.834           │
│ 420     │ 0.7029    │ 0.9095   │ 0.84            │
│ 421     │ 0.7013    │ 0.9074   │ 0.842           │
│ 423     │ 0.6986    │ 0.9078   │ 0.846           │
│ 426     │ 0.6956    │ 0

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.7656`, we are able to label at 93% accuracy and getting a completion rate of 74%. This means, we would ignore all the data points where confidence score is less than `0.7656` (which would end up being around 26% of all samples). This would, however, guarantee a very high quality labeled dataset for us. 